In [1]:
import numpy as np
import pandas as pd

In [245]:
df = pd.read_csv('TLM_LIST.csv')

.csv table view, first rows

In [246]:
df.head()

,Hex Addr,Byte,Package,Mnemonic,Short Description,Tlm Type,Tlm Conversion (EU/lsb),Eng. Units (EU)
0,0x420,32.0,COMMAND_TLM,CMD_STATUS,Command Status,uint8,1.0,0/OK 1/BAD_APID 2/BAD_OPCODE 3/BAD_DATA 7/NO_C...
1,0x421,33.0,COMMAND_TLM,CMD_REJECT_STATUS,Command Reject Status,uint8,1.0,0/OK 1/BAD_APID 2/BAD_OPCODE 3/BAD_DATA 7/NO_C...
2,0x422,34.0,COMMAND_TLM,CMD_ACCEPT_COUNT,Command Accept Count,uint8,1.0,none
3,0x423,35.0,COMMAND_TLM,CMD_REJECT_COUNT,Command Reject Count,uint8,1.0,none
4,0x424,36.0,COMMAND_TLM,LAST_ACC_CMD_BYTES1,Last Accept Cmd Bytes,uint8,1.0,none


Basic information about the data, two numerical tables, six with text

In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 8 columns):
Hex Addr                   724 non-null object
Byte                       724 non-null float64
Package                    724 non-null object
Mnemonic                   724 non-null object
Short Description          724 non-null object
Tlm Type                   724 non-null object
Tlm Conversion (EU/lsb)    724 non-null float64
Eng. Units (EU)            726 non-null object
dtypes: float64(2), object(6)
memory usage: 45.5+ KB


All integers

In [248]:
df['Tlm Type'].value_counts()

uint8     368
int32     117
int16     116
uint32     73
uint16     45
int8        5
Name: Tlm Type, dtype: int64

In [249]:
df[df['Package']=='TIME']

,Hex Addr,Byte,Package,Mnemonic,Short Description,Tlm Type,Tlm Conversion (EU/lsb),Eng. Units (EU)
24,0x441,65.0,TIME,TAI_SECONDS,TAI Seconds,uint32,0.2,sec
25,0x445,69.0,TIME,CYCLE_TIME,Cycle Time,uint32,0.2,sec
26,0x449,73.0,TIME,JD_OF_NOW_WRT_TAI,Julian Date TAI,uint32,1.0,day
27,0x44d,77.0,TIME,GPS_UPDATE_CYCLE_PERIOD,GPS Update Cycle Period,uint32,1.0,none
28,0x451,81.0,TIME,GPS_UPDATE_CYCLE_COUNT,GPS Update Cycle Count,uint32,1.0,none
29,0x455,85.0,TIME,TIME_VALID,Time Valid,uint8,1.0,1/YES 0/NO
30,0x456,86.0,TIME,CLOCK_SYNC_ENABLE,Clock Sync Enable,uint8,1.0,0/DS 1/EN
31,0x457,87.0,TIME,PPS_CONFIG,PPS Type,uint8,1.0,0/DS 1/GPS 2/INT


Managed to retrieve data as bytes, however I couldn't find correct encoding to see what exactly is written (I tried 'ascii', 'utf-8', 'itf-16' & 'latin-1') therefore I failed to split it and write it to an array based on csv structure so that I would be able to filter out the time, position etc.

Number of entries would be needed in the stk file so I calculated it based on provided data chunk size of 2068 bytes

In [253]:
import binascii
def bytes_from_file(filename, chunksize=2068):
    with open(filename, "rb") as f:
        while True:
            chunk = f.read(chunksize)
            if chunk:
                for b in chunk:
                    yield b   
            else:
                break

binary_data = []

for b in bytes_from_file('telemetry.bin'):
    binary_data.append(b)
    
#binary sample
print(binary_data[:100])

#number of lines based on data chunk provided
len(binary_data)/2068

[10, 0, 0, 7, 8, 1, 11, 231, 8, 195, 11, 217, 7, 158, 4, 228, 0, 0, 12, 239, 0, 153, 31, 219, 0, 0, 0, 0, 190, 239, 202, 254, 0, 0, 1, 0, 2, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 103, 50, 56, 55, 56, 95, 97, 32, 65, 0, 193, 0, 143, 101, 184, 62, 0, 0, 0, 5, 0, 37, 126, 26, 0, 0, 0, 150, 0, 0, 0, 145, 0, 1, 2, 0, 0, 0, 0, 143, 101, 184, 63, 0, 0, 0, 0]


3632.0

 I have chosen the ephemeris file format - 'EphemerisLLATimePos': 
 
 TimeInSeconds Lat Lon Alt
 
 but I could write only to .txt extension 

In [254]:
time = []
lat = []
long = []
alt = []

In [255]:
with open('stk.txt', 'w') as f:
    f.write('stk.v.5.0'+ '\n')
    f.write('BEGIN Ephemeris'+ '\n')
    f.write('NumberOfEphemerisPoints 3632'+ '\n')
    f.write('ScenarioEpoch           1 Jan 2000 00:00:00.000000000'+ '\n')
    f.write('InterpolationMethod     Lagrange'+ '\n')
    f.write('InterpolationOrder      1'+ '\n')
    f.write('DistanceUnit      Kilometers'+ '\n')
    f.write('CentralBody             Earth'+ '\n')
    f.write('CoordinateSystem        J2000'+ '\n')
    f.write('EphemerisLLATimePos'+ '\n')
    for index,value in enumerate(time):
        f.write('{} {} {} {}'.format(time[index], lat[index], lon[index], alt[index])+ '\n')
    f.write('END Ephemeris')

In [256]:
with open('stk.txt','r') as f:
    for line in f:
        print(line)

stk.v.5.0

BEGIN Ephemeris

NumberOfEphemerisPoints 3632

ScenarioEpoch           1 Jan 2000 00:00:00.000000000

InterpolationMethod     Lagrange

InterpolationOrder      1

DistanceUnit      Kilometers

CentralBody             Earth

CoordinateSystem        J2000

EphemerisLLATimePos

END Ephemeris
